In [1]:
import ipyvuetify as v

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import json
import requests
import ipywidgets as widgets
import numpy as np
from ipywidgets import Tab, Text, Layout, interact, widgets, HBox
from IPython.display import display, clear_output, HTML, display_html
import pandas as pd
from matplotlib.ticker import MaxNLocator
import math
from matplotlib.backends.backend_agg import FigureCanvasAgg
from handler import StatisticHandler
import datetime

## Equipazo presents: Inventory Tracking App

In [ ]:
#Data Fetching

In [ ]:
# Parts
all_parts_url = "https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/part"
all_parts_response = requests.get(all_parts_url)
all_parts_data = all_parts_response.json()
part_names = [part["name"] for part in all_parts_data]
part_prices = [part["price"] for part in all_parts_data]

#Parts by Supplier
parts_by_supplier_url = "http://127.0.0.1:5000/equipazo/supplier/1/parts/supplied"
parts_by_supplier_response = requests.get(parts_by_supplier_url)
parts_by_supplier_data = parts_by_supplier_response.json()

#Suppliers
all_suppliers_url= "http://127.0.0.1:5000/equipazo/supplier"
all_suppliers_response = requests.get(all_suppliers_url)
all_suppliers_data = all_suppliers_response.json()
supplier_ids= [supplier["supplier_id"] for supplier in all_suppliers_data]

#Transactions by warehouse; using user_id
trans_ware_url = "http://127.0.0.1:5000/equipazo/user/1/transactions"
trans_ware_response = requests.get(trans_ware_url)
trans_ware_data = trans_ware_response.json()

#Warerhouses
ware_url = "https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse"
ware_response = requests.get(ware_url)
ware_data = ware_response.json()
ware_ids= [ware["id"] for ware in ware_data]

#Parts by Warehouse
parts_ware_url = "http://127.0.0.1:5000/equipazo/warehouse/1/parts"
parts_ware_response = requests.get(parts_ware_url)
parts_ware_data = parts_ware_response.json()


#Users
all_user_url = "http://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/user"
all_user_response = requests.get(all_user_url)
all_user_data = all_user_response.json()
user_ids= [user["id"] for user in all_user_data]

# Loading Global Statistics Data
most_racks = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/most/rack').json()
most_incoming = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/most/incoming').json()
most_exchange_delivered = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/most/deliver').json()
most_transactions = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/most/transactions').json()
least_outgoing = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/least/outgoing').json()
most_trans_per_city = requests.get('https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/most/city').json()

colors=['#00A16B', '#008080','#D2F2D4', '#86C78A', '#C2F08C','#00794D', '#32CD32', '#40E0D0' ]

In [ ]:
#Drop down sorting menu for Part Prices

In [ ]:
part_sorting_dropdown = widgets.Dropdown(
    options={'Low to High': 1, 'High to Low': 2, 'A to Z': 3, 'Z to A': 4},
    value=1,
    description='Sort By:'
)

supplier_dropdown = widgets.Dropdown(
    options=supplier_ids,
    description='Select Supplier:',
    style={'description_width': 'initial'}  # Set description width to 'initial'
)

ware_dropdown = widgets.Dropdown(
    options=ware_ids,
    description='Select Warehouse:',
    style={'description_width': 'initial'}  # Set description width to 'initial'
)
dropdown_warehouse = widgets.Dropdown(
    options=[''], 
    description="Warehouse:", 
    layout=Layout(width='auto', display='flex')
)
statistics_options = ['Profit by Year',
                      'Top 5 Racks Below 25% Capacity', 
                      'Top 5 Most Expensive Racks', 
                      'Top 3 Supplier Transactions', 
                      'Lowest 3 Warehouse Parts by Type/Material', 
                      'Smallest Transaction Prices: Bottom 3 Days', 
                      'Top 3 Users with Highest Exchange Receipts']
dropdown_stats = widgets.Dropdown(
    options=statistics_options, 
    value=statistics_options[0], 
    description='Select Statistics:', 
    layout=Layout(width='auto', display='flex')
)
user_dropdown = widgets.Dropdown(
    options=user_ids, 
    description="User:", 
    style={'description_width': 'initial'}
)

# Global Statistics Dropdown
global_stat_select = widgets.Dropdown(
    options = {'Top 10 Warehouses with the Most Racks' : 1, 'Top 5 Warehouses with the Most Incoming Transactions' : 2,
    'Top 5 Warehouses that Delivered the Most Exchanges' : 3, 'Top 3 Users that Made the Most Transactions' : 4, 'Top 3 Warehouses with the Least Outgoing Transactions' : 5,
    'Top 3 Warehouse Cities with the Most Transactions' : 6},
    # value = 'Top 10 Warehouses with the Most Racks',
    description = 'Select Global Statistic to View:',
    style={'description_width': 'initial',
    'options_width' : 'initial'},
    # layout=Layout(width='auto', display='flex')
)

In [ ]:
#Content

In [ ]:
# Local Statistics
# URL to get user ID's
url_users_data = "https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/user"

# Store the selected warehouse ID
ware_id = None
user_id = None
stat = None
grey_color = "#D9DDDC"
shamrock_green = (64/255, 195/255, 148/255)

# Function to get the existing user IDs from the Heroku app
def get_existing_user_ids():
    try:
        user_response = requests.get(url_users_data)
        user_response.raise_for_status()  # Raise an exception for HTTP errors
        user_data_list = user_response.json()

        # Extract 'id' values and sort them
        user_ids = sorted([user['id'] for user in user_data_list])

        return user_ids
    except requests.exceptions.RequestException as e:
        print(f"Error fetching warehouse data: {e}")
        return []

def refresh_user_dropdown(change=None):
    user_ids = get_existing_user_ids()
    dropdown_user.options = user_ids
    dropdown_user.value = None  # Start with a blank option

    output_user_id.clear_output()

# Defining Options (usar esto si se ve muy desorganizado el dashboard)
statistics_options = ['Profit by Year',
                      'Top 5 Racks Below 25% Capacity', 
                      'Top 5 Most Expensive Racks', 
                      'Top 3 Supplier Transactions', 
                      'Lowest 3 Parts by Material', 
                      'Smallest Transaction Prices: Bottom 3 Days', 
                      'Top 3 Users with Highest Exchange Receipts']

# Create dropdown widgets
dropdown_user = widgets.Dropdown(options=[''], 
                                      description="User:", 
                                      layout=Layout(width='auto', display='flex'))
dropdown_stats = widgets.Dropdown(options=statistics_options, 
                                  value=statistics_options[1], 
                                  description='Select Statistics:', 
                                  layout=Layout(width='auto', display='flex'))
output_user_id = widgets.Output()
refresh_user_dropdown() 

# Watch for changes in the dropdown value and update ware_id and statistic
def on_dropdown_change(change):
    global user_id, stat
    user_id = dropdown_user.value
    stat = dropdown_stats.value
    with output_user_id:
        if user_id is not None:
            update_plot(user_id, stat)
            clear_output(wait=True)
        else:
            with output_user_id:
                print("Please choose a user.")
                clear_output(wait=True)
                plt.close('all')

dropdown_user.observe(on_dropdown_change, names='value')
dropdown_stats.observe(on_dropdown_change, names='value')
plt.close('all')

# Showing Dropdowns
html_workaround = HTML("<style>.widget-label { min-width: 150px !important; }</style>")
widgets_box = HBox([dropdown_user, dropdown_stats], layout=Layout(width='auto', flex_flow='row nowrap'))

# display(html_workaround)
# display(widgets_box, output_user_id)


def update_plot(user_id, stat):
    url_user_data = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/user/{user_id}"
    user_response = requests.get(url_user_data)
    user_response.raise_for_status()  # Raise an exception for HTTP errors
    user_data = user_response.json()
    
    # Get ware_id of selected user
    ware_id = user_data['wareid']
    html_content = f"<div style='text-align:center; font-size:20px;''><b>Warehouse {ware_id} | {stat}</b></div>"
    display(HTML(html_content))
    if stat == 'Profit by Year':
        url_profit = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/profit"
        data = get_data(url_profit)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_profit_chart(url_profit, stat, data, ware_id)
    elif stat == 'Top 5 Racks Below 25% Capacity':
        url_lowstock = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/rack/lowstock"
        data = get_data(url_lowstock)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:    
            plot_lowstock_chart(url_lowstock, stat, data, ware_id)
    elif stat == 'Top 5 Most Expensive Racks':
        url_expensive = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/rack/expensive"
        data = get_data(url_expensive)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_expensive_graph(url_expensive, stat, data, ware_id)
    elif stat == 'Top 3 Supplier Transactions':
        url_supp_trans = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/transaction/suppliers"
        data = get_data(url_supp_trans)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_supp_trans_graph(url_supp_trans, stat, data, ware_id)
    elif stat == 'Lowest 3 Parts by Material':
        url_part_material = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/rack/material"
        data = get_data(url_part_material)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_low_material(url_part_material, stat, data, ware_id)
    elif stat == 'Smallest Transaction Prices: Bottom 3 Days':
        url_leastcost = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/transaction/leastcost"
        data = get_data(url_leastcost)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_bottom_days(url_leastcost, stat, data, ware_id)
    elif stat == 'Top 3 Users with Highest Exchange Receipts':
        url_most_exchanges = f"https://equipazo-inventory-app-4a6e14d4919a.herokuapp.com/equipazo/warehouse/{ware_id}/users/receivesmost"
        data = get_data(url_most_exchanges)
        if "Error" in data:
            print("Not enough data to display statistic.")
        else:
            plot_top_exchange_users(url_most_exchanges, stat, data, ware_id)


# Defining functions to display the different stat charts/graphs/tables

# Function to update the plot based on the profit data
def plot_profit_chart(url, stat, data, ware_id):
    display(create_tables(url, stat))
    years = [entry['year'] for entry in data]
    profits = [entry['profit'] for entry in data]
    
    if len(years) > 0 and len(profits) > 0:
        plt.figure(figsize=(4, 4))
        plt.bar(years, profits, color=shamrock_green)
        plt.title("Profit by Year")
        plt.xlabel("Year")
        plt.ylabel("Profit") 
        plt.show()
    else:
        print(f"No valid data to plot for warehouse {ware_id}")

def plot_lowstock_chart(url, stat, data, ware_id):
    display(create_tables(url, stat))
    # Sorting data by capacity in descending order
    sorted_data = sorted(data, key=lambda x: float(x['capacity_threshold']), reverse=True)
    # Extracting values for plotting
    rack_values = [float(entry['capacity_threshold']) * 100 for entry in sorted_data]
    racks = [f"Rack {entry['rack_id']}" for entry in sorted_data]
    ranks = range(1, len(sorted_data) + 1)

    # Dictionary to store pie chart images
    pie_charts = {}
    
    # Plotting the pie charts
    for rank, (rack_value, rack) in enumerate(zip(rack_values, racks), start=1):
        fig_ls, ax_ls = plt.subplots()
        ax_ls.pie([rack_value, 100 - rack_value], labels=['Occupied', 'Empty'],
               autopct='%1.1f%%', startangle=90, colors=[shamrock_green, grey_color])
        # ax_ls.set_title(f'Rack {rank} - {rack_value:.2f}% Occupied')
        # Save the pie chart image in the dictionary
        canvas = FigureCanvasAgg(fig_ls)
        canvas.draw()
        # pie_charts[rank] = np.array(canvas.renderer.buffer_rgba())
        pie_charts[rank] = [rack, np.array(canvas.renderer.buffer_rgba())]

    # Create empty pie charts for remaining slots
    empty_ranks = range(len(sorted_data) + 1, 6)
    while len(pie_charts) < 5:
        empty_rank = len(pie_charts) + 1
        fig_lsp, axs_lsp = plt.subplots()
        axs_lsp.pie([100, 0], labels=['Empty', ''],
               autopct='%1.1f%%', startangle=90, colors=[grey_color, 'white'])
        # axs_lsp.set_title(f'Rack {empty_rank} - 0.00% Occupied')
        # Save the empty pie chart image in the dictionary
        canvas = FigureCanvasAgg(fig_lsp)
        canvas.draw()
        pie_charts[empty_rank] = [-1, np.array(canvas.renderer.buffer_rgba())]

    # Display pie chart images side by side
    fig_lsp, axs_lsp = plt.subplots(1, 5, figsize=(15, 5))
    plt.subplots_adjust(wspace=0.1, hspace=0.04)

    # This for loop is what isn't working
    for fig_rank, (rack_id, rgba_data) in pie_charts.items():
        # Display the RGBA image using AxesImage
        axs_lsp[fig_rank - 1].imshow(rgba_data)
        if 1 <= fig_rank <= len(rack_values):
            if rack_values[fig_rank - 1] > 0:
                axs_lsp[fig_rank - 1].set_title(f'#{fig_rank}\n{rack_id}\n{rack_values[fig_rank - 1]:.2f}% Occupied')
        else:
            axs_lsp[fig_rank - 1].set_title(f'No more racks available.')

        axs_lsp[fig_rank - 1].axis('off')  # Turn off axis
    # Display the figures directly in the notebook
    display(fig_lsp)
    
def plot_expensive_graph(url, stat, data, ware_id):
    # Display data table
    display(create_tables(url, stat))
    #Sort data
    sorted_data = sorted(data, key=lambda x: x['rack_value'], reverse=True)
    # Call function to plot car chart
    plot_rank_chart(sorted_data, 'rack_id', 'rack_value', 5)

def plot_supp_trans_graph(url, stat, data, ware_id):
    # Display data table
    display(create_tables(url, stat))
    #Sort data
    sorted_data = sorted(data, key=lambda x: x['trans_count'], reverse=True)
    # Call function to plot car chart
    plot_rank_chart(sorted_data, 'supplier_id', 'trans_count', 3)

def plot_low_material(url, stat, data, ware_id):
    # Display data table
    display(create_tables(url, stat))
    #Sort data
    sorted_data = sorted(data, key=lambda x: x['total_stock'], reverse=False)
    # Call function to plot car chart
    plot_rank_chart(sorted_data, 'part_type', 'total_stock', 3)

def plot_bottom_days(url, stat, data, ware_id):
    # Display data table
    display(create_tables(url, stat))
    #Sort data
    sorted_data = sorted(data, key=lambda x: x['trans_cost'], reverse=False)
    # Call function to plot car chart
    plot_rank_chart(sorted_data, 'trans_date', 'trans_cost', 3)

def plot_top_exchange_users(url, stat, data, ware_id):
    # Display data table
    display(create_tables(url, stat))
    #Sort data
    sorted_data = sorted(data, key=lambda x: x['receipt_amount'], reverse=False)
    # Call function to plot car chart
    plot_rank_chart(sorted_data, 'user_id', 'receipt_amount', 3)
     
def get_data(url):
    try:
        response = requests.post(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data_list = response.json()
        return data_list
    except requests.exceptions.RequestException as e:
        return f"Error fetching data: {e}"

def create_tables(url, stat):
    # Get data
    list = get_data(url)
    df = pd.DataFrame(list)
    df.columns = [col.replace('_', ' ').title() for col in df.columns]
    df.index = range(1, len(df) + 1) 
    styled_df = df.style.set_table_styles([{'selector': '', 'props': [('border', '1px solid black')]}])
    return styled_df

def plot_rank_chart(sorted_data, x_label, y_label, num_ranks):
    # Extracting values for plotting
    x_values = [f"{entry[x_label]}" for entry in sorted_data[:num_ranks]]
    y_values = [entry[y_label] for entry in sorted_data[:num_ranks]]
    ranks = range(1, len(sorted_data) + 1)
    # Create a new figure and axes for the bar graph
    fig_rc, ax_rc = plt.subplots(figsize=(8, 6))

    # Plotting the bar graph
    bars = ax_rc.bar(ranks, y_values, color=shamrock_green)

    if y_label == 'receipt_amount':
        new_y_label = f'({y_label.replace("_", " ").title()})'
    else:
        new_y_label = f'({y_label.split("_")[1].title()})'
        
    new_x_label = x_label.split("_")[1].title()

    ax_rc.yaxis.set_label_coords(-0.07, 1.04)
    ax_rc.set_ylabel(new_y_label, rotation=0, labelpad=-15,ha='left', va='top')
    ax_rc.set_xlabel('Rank')
    ax_rc.set_xticks(range(1, num_ranks + 1))

    width = ax_rc.get_ylim()[1] - ax_rc.get_ylim()[0]
    ax_rc.set_ylim(0, max(y_values) + 1)
    
    # Add "Label" text on top of their respective bars
    for rank, label in zip(ranks, x_values):
        if x_label == 'trans_date':
            date_object = datetime.datetime.strptime(label, '%a, %d %b %Y %H:%M:%S %Z')
            new_label = date_object.strftime('%a, %d %b %Y')
        else:
            new_label = label
        bar = bars[rank - 1]
        ax_rc.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{new_x_label}:{new_label}', ha='center', va='bottom', rotation=45, color='black')

    ax_rc.tick_params(axis='both', which='both', length=5, color='black', pad=5)
    ax_rc.grid(axis='y', color='#999DA0', linestyle='--', linewidth=0.8)
    ax_rc.spines['top'].set_visible(False)
    ax_rc.spines['right'].set_visible(False)

    plt.show()
    
def extract_years_profits(url, ware_id):
    try:
        profit_response = requests.post(url)
        profit_response.raise_for_status()  # Raise an exception for HTTP errors
        profit_list = profit_response.json()

        if type(profit_list) is str:
            print(f"Error: {profit_list}")
            return 0,0
        else:
            years = [entry['year'] for entry in data]
            profits = [entry['profit'] for entry in data]
            return years, profits

    except requests.exceptions.RequestException as e:
        print(f"Error fetching profit data for warehouse {ware_id}: {e}")
        return 0, 0

In [ ]:
# Global Statistics
# Formatting Global Stat Data
# Top 10 Most Racks
most_racks_ware_id = [f'Warehouse {data["ware_id"]}' for data in most_racks]
most_racks_rack_count = [data["rack_count"] for data in most_racks]
# Top 5 Most Incoming
most_incoming_ware = [f'Warehouse {data["ware_id"]}' for data in most_incoming]
most_incoming_count = [data["inc_transaction_count"] for data in most_incoming]
# Top 5 Exchange Delivered
most_exchange_ware = [f'Warehouse {data["ware_id"]}' for data in most_exchange_delivered]
most_exchange_count = [data["delivered_exchanges"] for data in most_exchange_delivered]
# Top 3 Users Trans
most_trans_users = [f'User {data["user_id"]}' for data in most_transactions]
most_trans_count = [data["trans_count"] for data in most_transactions]
# Top 3 Least Outgoing
least_out_ware = [f'Warehouse {data["ware_id"]}' for data in least_outgoing]
least_out_count = [data["outgoing_trans_count"] for data in least_outgoing]
# Top 3 Cities Trans
top_cities = [data["city"] for data in most_trans_per_city]
top_cities_trans = [data["city_trans_count"] for data in most_trans_per_city]
 
# Create Subplot for Global Stats
fig_gs, ax_gs = plt.subplots(figsize=(8, 6))

# Output Widget for Global Stat Bar Chart
global_output = widgets.Output()
with global_output:
    plt.show()

# Load Statistics Data - Bar Chart
def load_global(selected):
    if selected == 1:
        xdata = most_racks_ware_id 
        ydata = most_racks_rack_count 
        title = 'Top 10 Warehouses with the Most Racks'
        ylabel = 'Number of Racks'
        xlabel = 'Warehouses'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    elif selected == 2:
        xdata = most_incoming_ware
        ydata = most_incoming_count
        title = 'Top 5 Warehouses with the Most Incoming Transactions'
        ylabel = 'Number of Incoming Transactions'
        xlabel = 'Warehouses'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    elif selected == 3:
        xdata = most_exchange_ware
        ydata = most_exchange_count
        title = 'Top 5 Warehouses that Delivered the Most Exchanges'
        ylabel = 'Number of Exchanges Delivered'
        xlabel = 'Warehouses'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    elif selected == 4:
        xdata = most_trans_users
        ydata = most_trans_count
        title = 'Top 3 Users that Made the Most Transactions'
        ylabel = 'Number of Transactions Made'
        xlabel = 'Users'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    elif selected == 5:
        xdata = least_out_ware
        ydata = least_out_count
        title = 'Top 3 Warehouses with the Least Outgoing Transactions'
        ylabel = 'Number of Outgoing Transactions'
        xlabel = 'Warehouses'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    elif selected == 6:
        xdata = top_cities
        ydata = top_cities_trans
        title = 'Top 3 Warehouse Cities with the Most Transactions'
        ylabel = 'Number of Transactions'
        xlabel = 'Cities'
        return render_global(xdata, ydata, title, xlabel, ylabel)
    
# Bar Chart Layout 
def render_global(xdata, ydata, title, xlabel, ylabel):
    with global_output:
        # Clear previous data
        ax_gs.clear()
        # Layout
        bars = ax_gs.bar(xdata, ydata, align='center')
        ax_gs.grid(axis='y', linestyle='--', alpha=0.8)
        # Adjust labels to fit for many columns
        if len(xdata) > 5:
            ax_gs.set_xticks(xdata, xdata, rotation=45, ha='right')
        # Set label names and chart title
        ax_gs.set_xlabel(xlabel)
        ax_gs.set_ylabel(ylabel)
        ax_gs.set_title(title)
        # Place count at the end of each bar
        for bar, count in zip(bars, ydata):
            ax_gs.text(bar.get_x() + bar.get_width()/2 , bar.get_height(), count, ha='center', va='bottom', fontsize=10)
        plt.tight_layout()
        # display(fig)

# Attach event handler to the dropdown
global_stat_select.observe(lambda change: load_global(change.new), names='value')
# Initial Stat for Bar Chart
load_global(global_stat_select.value)

# Load Statistics Data - Table
def global_table(selected):
    with global_table_out:
        clear_output(wait=True)  # Clear the output area
        # Load Data for selected statistic
        if selected == 1:
            data = most_racks
            table_dict = {'ware_id' : 'Warehouse ID', 'rack_count' : 'Number of Racks'}
            col = ['Warehouse ID', 'Number of Racks']
        elif selected == 2:
            data = most_incoming
            table_dict = {'ware_id' : 'Warehouse ID', 'inc_transaction_count' : 'Number of Incoming Transactions'}
            col = ['Warehouse ID', 'Number of Incoming Transactions']
        elif selected == 3:
            data = most_exchange_delivered
            table_dict = {'ware_id' : 'Warehouse ID', 'delivered_exchanges' : 'Number of Exchanges Delivered'}
            col = ['Warehouse ID', 'Number of Exchanges Delivered']
        elif selected == 4:
            data = most_transactions
            table_dict = {'user_id' : 'User ID', 'trans_count' : 'Number of Transactions Made'}
            col = ['User ID', 'Number of Transactions Made']
        elif selected == 5:
            data = least_outgoing
            table_dict = {'ware_id' : 'Warehouse ID', 'outgoing_trans_count' : 'Number of Outgoing Transactions'}
            col = ['Warehouse ID', 'Number of Outgoing Transactions']
        elif selected == 6:
            data = most_trans_per_city
            table_dict = {'city' : 'City', 'city_trans_count' : 'Number of Transactions'}
            col = ['City', 'Number of Transactions']
        # Create table with dataframe
        global_df = pd.DataFrame(data)
        global_df = global_df.rename(columns=table_dict)
        global_df = global_df[col]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(global_df)
global_table_out = widgets.Output()
with global_table_out:
    global_table(global_stat_select.value)
global_stat_select.observe(lambda change: global_table(change.new), names='value')

# global_stat_select
# global_table_out
# global_output



In [ ]:
#Tab pannel 1

In [ ]:
# Tab contents
tab_contents = [widgets.Text(description=name) for name in ['Local Statistics', 'Global Statistics']]

# Layout for Global Statistics Tab
layout_global_stat = widgets.HBox()
layout_global_stat.children = [global_table_out, global_output]

tab_contents[0] = widgets.VBox([widgets_box, output_user_id])
tab_contents[1] = widgets.VBox([global_stat_select, layout_global_stat])


# Configure the tab widget
tab = widgets.Tab()
tab.children = tab_contents
tab.titles = ['Local Statistics', 'Global Statistics']

# Display the tab
display(tab)

In [ ]:
# Bar chart for Part Prices
fig_pchart, ax_pchart = plt.subplots(figsize=(7.5, 6))
bars=ax_pchart.barh(part_names, part_prices, color=colors[5])
ax_pchart.grid(axis='x', linestyle='--', alpha=0.8)
ax_pchart.set_xlabel('Part Price')
ax_pchart.set_ylabel('Part Name')
ax_pchart.set_title('Bar Chart of Part Prices')
plt.tight_layout()
# plt.yticks(ha='right',  fontsize=5)
for bar, price in zip(bars, part_prices):
    ax_pchart.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2, f"  ${price:.2f}", ha='left', va='center', fontsize=6)

chart_output = widgets.Output()
with chart_output:
    plt.show()

# Update chart based on sorting selection for Part Prices
def update_parts_chart(sort_order):
    # Sort data based on selected order
    if sort_order == 1:
        all_parts_data.sort(key=lambda x: x['price'], reverse=True)
    elif sort_order == 2:
        all_parts_data.sort(key=lambda x: x['price'])
    elif sort_order == 3:
        all_parts_data.sort(key=lambda x: x['name'], reverse=True)
    elif sort_order == 4:
        all_parts_data.sort(key=lambda x: x['name'])

    # Update chart with sorted data
    with chart_output:
        part_names = [part["name"] for part in all_parts_data]
        part_prices = [part["price"] for part in all_parts_data]
        ax_pchart.clear() # Clear existing data
        bars=ax_pchart.barh(part_names, part_prices, color=colors[5])
        ax_pchart.grid(axis='x', linestyle='--', alpha=0.8)
        ax_pchart.set_xlabel('Part Price')
        ax_pchart.set_ylabel('Part Name')
        ax_pchart.set_title('Bar Chart of Part Prices')
        ax_pchart.tick_params(labelsize=5)
        for bar, price in zip(bars, part_prices):
            ax_pchart.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2, f"  ${price:.2f}", ha='left', va='center', fontsize=6)
        plt.tight_layout()

part_sorting_dropdown.observe(lambda change: update_parts_chart(change.new), names='value')
update_parts_chart(part_sorting_dropdown.value)

# display(chart_output)

# Part Prices Histogram
fig, ax_price_hist = plt.subplots(figsize=(8, 6))
ax_price_hist.hist(part_prices, bins=9, edgecolor='black', color=colors[3])
ax_price_hist.set_xlabel('Part Price ($)')
ax_price_hist.set_ylabel('Frequency')
ax_price_hist.set_title('Histogram of Part Prices')
ax_price_hist.grid(axis='x', linestyle='--', alpha=0.7)

price_hist_output = widgets.Output()
with price_hist_output:
    plt.show()



#Table for Parts Supplied by Supplier
def display_parts_table():
    if parts_by_supplier_data == "Supplier ID does not exist or Supplier does not supply parts yet":
        clear_output(wait=True)  # Clear the output area
        return print("\nSupplier does not supply parts yet")
    else:
        parts_df = pd.DataFrame(parts_by_supplier_data)
        parts_df = parts_df.rename(columns={"part_id": "Part ID", "part_name": "Part Name", "part_price": "Part Price", "part_type": "Part Type", "stock": "Stock"})
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(parts_df)
parts_table = widgets.Output()
with parts_table:
    display_parts_table()

# Update Parts Supplied by Supplier table based on Supplier ID
def update_supplier_table(sup_id):
    with parts_table:
        clear_output(wait=True)  # Clear the output area
        parts_by_supplier_url = f"http://127.0.0.1:5000/equipazo/supplier/{sup_id}/parts/supplied"
        parts_by_supplier_response = requests.get(parts_by_supplier_url)
        parts_by_supplier_data = parts_by_supplier_response.json()
        if parts_by_supplier_data == "Supplier ID does not exist or Supplier does not supply parts yet":
            clear_output(wait=True)  # Clear the output area
            return print("\nSupplier does not supply parts yet")
        else:
            parts_df = pd.DataFrame(parts_by_supplier_data)
            parts_df = parts_df.rename(columns={"part_id": "Part ID", "part_name": "Part Name", "part_price": "Part Price", "part_type": "Part Type", "stock": "Stock"})
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):
                display(parts_df)
supplier_dropdown.observe(lambda change: update_supplier_table(change.new), names='value')
update_supplier_table(supplier_dropdown.value)

#Parts Supplied by Supplier pie chart
part_name_sup = [part_sup["part_name"] for part_sup in parts_by_supplier_data]
stock = [part_sup["stock"] for part_sup in parts_by_supplier_data]
fig, sup_pchart = plt.subplots()
sup_pchart.pie(stock, labels=part_name_sup, autopct='%1.1f%%', colors=colors)

sup_pchart_output = widgets.Output()
with sup_pchart_output:
    plt.show()

# Update Parts Supplied by Supplier pie chart based on Supplier ID
def update_supplier_pchart(sup_id):
    sup_pchart_output.clear_output(wait=True)  # Clear existing data
    with sup_pchart_output:
        parts_by_supplier_url = f"http://127.0.0.1:5000/equipazo/supplier/{sup_id}/parts/supplied"
        parts_by_supplier_response = requests.get(parts_by_supplier_url)
        parts_by_supplier_data = parts_by_supplier_response.json()
        if parts_by_supplier_data == "Supplier ID does not exist or Supplier does not supply parts yet":
            sup_pchart.clear()  # Clear existing data
            sup_pchart.pie([1], labels=["None"], colors=["grey"])
            sup_pchart.set_alpha(0)  # Make slices invisible
            sup_pchart.text(0.5, 0.5, "No data available", ha='center', va='center', fontsize=12)
        else:
            part_name_sup = [part_sup["part_name"] for part_sup in parts_by_supplier_data]
            stock = [part_sup["stock"] for part_sup in parts_by_supplier_data]
            if sum(stock) == 0:
                sup_pchart.clear()  # Clear existing data
                sup_pchart.pie([1], labels=["None"], colors=["grey"])
                sup_pchart.set_alpha(0)  # Make slices invisible
                sup_pchart.text(0.5, 0.5, "Supplier is out of supplies", ha='center', va='center', fontsize=12)
            else:
                sup_pchart.clear()  # Clear existing data
                sup_pchart.pie(stock, labels=part_name_sup, autopct='%1.1f%%', colors=colors)
supplier_dropdown.observe(lambda change: update_supplier_pchart(change.new), names='value')
update_supplier_pchart(supplier_dropdown.value)


# Transactions in a Warehouse Table
def display_trans_ware_table():
    if trans_ware_data == "This warehouse does not have transactions":
        clear_output(wait=True)  # Clear the output area
        return print("\n", trans_ware_data)
    else:
        trans_ware_df = pd.DataFrame(trans_ware_data)
        trans_ware_df = trans_ware_df.rename(columns={"rack_id": "Rack ID", "trans_date": "Date", "trans_id": "Transaction ID", "trans_type": "Type", "ware_id":"Warehouse ID"})
        html_style = "<div style='height: 400px; overflow: auto; width: fit-content'>"
        display(HTML(html_style + trans_ware_df.to_html() + "</div>"))
        # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
trans_ware_table = widgets.Output()
with trans_ware_table:
    display_trans_ware_table()

#Update Transactions in a Warehouse table based on Warehouse ID
def update_trans_ware_table(user_id):
    with trans_ware_table:
        clear_output(wait=True)  # Clear the output area
        trans_ware_url = f"http://127.0.0.1:5000/equipazo/user/{user_id}/transactions"
        trans_ware_response = requests.get(trans_ware_url)
        trans_ware_data = trans_ware_response.json()
        if trans_ware_data == "This warehouse does not have transactions":
            clear_output(wait=True)  # Clear the output area
            return print("\n", trans_ware_data)
        else:
            trans_ware_df = pd.DataFrame(trans_ware_data)
            trans_ware_df = trans_ware_df.rename(columns={"rack_id": "Rack ID", "trans_date": "Date", "trans_id": "Transaction ID", "trans_type": "Type", "ware_id":"Warehouse ID"})
            html_style = "<div style='height: 400px; overflow: auto; width: fit-content'>"
            display(HTML(html_style + trans_ware_df.to_html() + "</div>"))
            # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
user_dropdown.observe(lambda change: update_trans_ware_table(change.new), names='value')
update_trans_ware_table(user_dropdown.value)



#Transaction in a Warehouse Bar Chart
fig_trans_ware, ax_trans_ware = plt.subplots(figsize=(8, 6))

tran_ware_type = [tran["trans_type"] for tran in trans_ware_data]
tran_types_count = pd.Series(tran_ware_type).value_counts()

ax_trans_ware.bar(tran_types_count.index, tran_types_count.values, color='green')
ax_trans_ware.grid(axis='y', linestyle='--', alpha=0.8)
ax_trans_ware.set_xlabel('Transaction Type')
ax_trans_ware.set_ylabel('Number of Transaction Made')
ax_trans_ware.set_title('Transactions in Warehouse')
plt.tight_layout()
# plt.yticks(ha='right',  fontsize=5)

trans_ware_chart_output = widgets.Output()
with trans_ware_chart_output:
    plt.show()

# Update Transaction in a Warehouse Bar Chart
def update_trans_ware_chart(user_id):
    with trans_ware_chart_output:
        trans_ware_url = f"http://127.0.0.1:5000/equipazo/user/{user_id}/transactions"
        trans_ware_response = requests.get(trans_ware_url)
        trans_ware_data = trans_ware_response.json()
        if trans_ware_data == "This warehouse does not have transactions":
            ax_trans_ware.clear() # Clear the output area
            return
        else:
            tran_ware_type = [tran["trans_type"] for tran in trans_ware_data]
            tran_types_count = pd.Series(tran_ware_type).value_counts()
            ax_trans_ware.clear() # Clear existing data
            ax_trans_ware.bar(tran_types_count.index, tran_types_count.values, color='green')
            ax_trans_ware.grid(axis='y', linestyle='--', alpha=0.8)
            ax_trans_ware.set_xlabel('Transaction Type')
            ax_trans_ware.set_ylabel('Number of Transaction Made')
            ax_trans_ware.set_title('Transactions in Warehouse')
            plt.tight_layout()
user_dropdown.observe(lambda change: update_trans_ware_chart(change.new), names='value')
update_trans_ware_chart(user_dropdown.value)


#Update Parts in a Warehouse
def update_parts_ware_table(wid):
    with parts_ware_out:
        clear_output(wait=True)  # Clear the output area
        parts_ware_url = f"http://127.0.0.1:5000/equipazo/warehouse/{wid}/parts"
        parts_ware_response = requests.get(parts_ware_url)
        parts_ware_data = parts_ware_response.json()
        if parts_ware_data == "This warehouse does not have parts":
            clear_output(wait=True)  # Clear the output area
            return print("\n",parts_ware_data)
        else:
            table_dict = {"part_id": "Part ID", "part_name": "Part Name", "part_price": "Part Price ($)", "part_type": "Part Type"}
            col = ['Part ID', 'Part Name', 'Part Price ($)', 'Part Type']
            parts_ware_df = pd.DataFrame(parts_ware_data)
            parts_ware_df = parts_ware_df.rename(columns=table_dict)
            parts_ware_df = parts_ware_df[col]
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):
                display(parts_ware_df)
parts_ware_out = widgets.Output()
with parts_ware_out:
    update_parts_ware_table(ware_dropdown.value)
ware_dropdown.observe(lambda change: update_parts_ware_table(change.new), names='value')      


#Parts in Warehouse bar chart
part_type_ware = [part_ware["part_type"] for part_ware in parts_ware_data]
part_ware_count = pd.Series(part_type_ware).value_counts()

fig_part_ware, part_ware_bar = plt.subplots(figsize=(8, 6))
bars=part_ware_bar.bar(part_ware_count.index, part_ware_count.values, color=colors[0], edgecolor='black')
part_ware_bar.grid(axis='y', linestyle='--', alpha=1)
part_ware_bar.set_xlabel('Part Type')
part_ware_bar.set_ylabel('Amount of Types')
part_ware_bar.set_title('Parts in Warehouse')
part_ware_bar.yaxis.set_major_locator(MaxNLocator(integer=True))
max_y = part_ware_count.max()
max_y = math.ceil(max_y)
part_ware_bar.set_ylim(0, max_y)
for bar, count in zip(bars, part_ware_count.values):
    part_ware_bar.text(bar.get_x() + bar.get_width()/2 , bar.get_height(), count, ha='center', va='bottom', fontsize=10)
plt.tight_layout()

part_ware_bar_output = widgets.Output()
with part_ware_bar_output:
    plt.show()

#Update Parts in Warehouse chart
def update_part_ware_chart(wid):
    with part_ware_bar_output:
        clear_output(wait=True)  # Clear the output area
        parts_ware_url = f"http://127.0.0.1:5000/equipazo/warehouse/{wid}/parts"
        parts_ware_response = requests.get(parts_ware_url)
        parts_ware_data = parts_ware_response.json()
        
        part_type_ware = [part_ware["part_type"] for part_ware in parts_ware_data]
        part_ware_count = pd.Series(part_type_ware).value_counts()

        part_ware_bar.clear()  # Clear existing plot
        bars = part_ware_bar.bar(part_ware_count.index, part_ware_count.values, color=colors[0], edgecolor='black')
        part_ware_bar.grid(axis='y', linestyle='--', alpha=1)
        part_ware_bar.set_xlabel('Part Type')
        part_ware_bar.set_ylabel('Amount of Parts')
        part_ware_bar.set_title('Parts in Warehouse')
        
        part_ware_bar.yaxis.set_major_locator(MaxNLocator(integer=True))
        for bar, count in zip(bars, part_ware_count.values):
            part_ware_bar.text(bar.get_x() + bar.get_width()/2 , bar.get_height(), count, ha='center', va='bottom', fontsize=10)
        plt.tight_layout()
ware_dropdown.observe(lambda change: update_part_ware_chart(change.new), names='value')
update_part_ware_chart(ware_dropdown.value)


In [ ]:
#Tab Section 2

In [ ]:
# Tab contents
# tab_contents = [widgets.Text(description=name) for name in ['Local Statistics', 'Global Statistics', 'Transactions', 'Parts Prices', 'Parts by Supplier', 'Parts in Warehouses']]
tab_contents = [widgets.Text(description=name) for name in ['Transactions', 'Parts Prices', 'Parts by Supplier', 'Parts in Warehouses']]

# Layout for bar chart and histogram
layout_parts_prices = widgets.HBox()
layout_parts_prices.children = [chart_output, price_hist_output]

layout_parts_sup = widgets.HBox()
layout_parts_sup.children = [parts_table, sup_pchart_output]

layout_trans_ware= widgets.HBox()
layout_trans_ware.children = [trans_ware_table, trans_ware_chart_output]

layout_parts_ware= widgets.HBox()
layout_parts_ware.children = [parts_ware_out, part_ware_bar_output]

# Chart outputs to respective tabs
tab_contents[0] = widgets.VBox([user_dropdown, layout_trans_ware])
tab_contents[1] = widgets.VBox([part_sorting_dropdown, layout_parts_prices])
tab_contents[2] = widgets.VBox([supplier_dropdown, layout_parts_sup])
tab_contents[3] = widgets.VBox([ware_dropdown, layout_parts_ware])

# Configure the tab widget
tab = widgets.Tab()
tab.children = tab_contents
# tab.titles = ['Local Statistics', 'Global Statistics', 'Transactions', 'Parts Prices', 'Parts by Supplier', 'Parts in Warehouses']
tab.titles = ['Transactions', 'Parts Prices', 'Parts by Supplier', 'Parts in Warehouses']
# Display the tab
display(tab)

In [ ]:
# np.random.seed(0)

# n_bins = 10
# x = np.random.randn(1000, 3)

# fig, axes = plt.subplots(nrows=2, ncols=2)
# ax0, ax1, ax2, ax3 = axes.flatten()

# part_names = [part["name"] for part in all_parts_data]
# part_prices = [part["price"] for part in all_parts_data]

# # ax0.figure(figsize=(12, 16))
# ax0.barh(part_names, part_prices, color='blue')
# ax0.grid(axis='x', linestyle='--', alpha=1)
# # ax0.xlabel('Part Price')
# # ax0.ylabel('Part Name')
# # ax0.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
# ax0.set_title('Part Prices')

# ax1.hist(x, n_bins, density=1, histtype='bar', stacked=True)
# ax1.set_title('stacked bar')

# ax2.hist(x, n_bins, histtype='step', stacked=True, fill=False)
# ax2.set_title('stack step (unfilled)')

# # Make a multiple-histogram of data-sets with different length.
# x_multi = [np.random.randn(n) for n in [10000, 5000, 2000]]
# ax3.hist(x_multi, n_bins, histtype='bar')
# ax3.set_title('different sample sizes')

# fig.tight_layout()
# plt.show()